In [34]:
import pandas as pd
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.metadata import Metadata




In [ ]:
df = pd.read_csv('''Directorio''')

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4552 entries, 0 to 4551
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha      4552 non-null   datetime64[ns]
 1   Importe    4552 non-null   float64       
 2   Categoria  4552 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 106.8+ KB


In [37]:
df["Fecha"] = pd.to_datetime(df["Fecha"])
#df["Categoria"] = df["Categoria"].astype("category")


In [39]:
from sdv.metadata import SingleTableMetadata

# Configuración de los metadatos
metadata = SingleTableMetadata()

# Detectar columnas automáticamente
metadata.detect_from_dataframe(data=df)

# Si es necesario, agrega o actualiza columnas manualmente
metadata.update_column('Fecha', sdtype='datetime')
metadata.update_column('Importe', sdtype='numerical')
metadata.update_column('Categoria', sdtype='categorical')

# Verifica los metadatos
print(metadata.to_dict())



{'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1', 'columns': {'Fecha': {'sdtype': 'datetime'}, 'Importe': {'sdtype': 'numerical'}, 'Categoria': {'sdtype': 'categorical'}}}


In [ ]:
synthesizer = CTGANSynthesizer(metadata, epochs=10000, discriminator_steps=3)
synthesizer.fit(df)

synthetic_data = synthesizer.sample(4500)  # Generar 4500 filas sintéticas

#5000 - 71 minutos
#10000 - 152.80 minutos
#10000, 3 steps - 385.21

c:\Users\diego\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
c:\Users\diego\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [ ]:
synthetic_data.to_csv('''Directorio''', index=False)

In [42]:
synthetic_data

,Fecha,Importe,Categoria
0,2023-11-17,-11.89,impuestos
1,2024-07-30,-19.36,comida_fuera
2,2024-06-17,-9.18,mascota
3,2024-09-26,-2.53,comida
4,2019-06-04,-44.04,movimiento
...,...,...,...
4495,2021-08-25,-25.89,servicios
4496,2021-10-04,12.98,ingresos
4497,2022-11-16,0.44,comida
4498,2023-05-16,154.74,ingresos


In [43]:
from sdv.evaluation.single_table import run_diagnostic, evaluate_quality
from sdv.evaluation.single_table import get_column_plot

# 1. perform basic validity checks
diagnostic = run_diagnostic(df, synthetic_data, metadata)

# 2. measure the statistical similarity
quality_report = evaluate_quality(df, synthetic_data, metadata)
   


Generating report ...

(1/2) Evaluating Data Validity: |          | 0/3 [00:00<?, ?it/s]|

(1/2) Evaluating Data Validity: |██████████| 3/3 [00:00<00:00, 180.78it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 141.40it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 3/3 [00:00<00:00, 30.70it/s]|
Column Shapes Score: 85.14%

(2/2) Evaluating Column Pair Trends: |██████████| 3/3 [00:00<00:00, 41.31it/s]|
Column Pair Trends Score: 54.49%

Overall Score (Average): 69.82%



In [44]:
quality_report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,Fecha,KSComplement,0.972886
1,Importe,KSComplement,0.904850
2,Categoria,TVComplement,0.676350
